![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 
<font size=4>   </font>
</div>


<div align="center"> 
<font size=5> Ejercicio 3 - Micro-óptica </font>
</div>

- **Fecha**: 
        
- **Alumno**: Alex Recuenco (Alejandro G Recuenco)

In [ ]:
from matplotlib import rcParams

rcParams["figure.dpi"] = 250

from matplotlib import pyplot as plt
from diffractio.scalar_masks_XZ import Scalar_mask_XZ
from diffractio.utils_drawing import draw_several_fields


def draw_intensity_phase_XY(field: Scalar_mask_XZ, title: str):
    # Bug: Draw_several_fields only works for XY
    draw_several_fields(
        fields=(field, field), kinds=["intensity", "phase"], title=title, logarithm=1e2
    )
    axes = plt.gca()
    axes.figure._suptitle.set_fontsize(12)

    plt.show()

# Introducción
En estos ejercicios nos vamos a centrar en elementos microópticos como lentes de Fresnel y redes de difraccion.

# Enunciados

1. Sea una lente estándar de focal f′=10 mm y diámetro D=0.5 mm.

 a. Determine, en el marco XY y utilizando la aproximación de elemento delgado, la distribución de intensidad en el plano focal, mediante el modelo de Rayleigh-Sommerfeld.  Determine, a partir de esta imagen, el perfil radial de intensidad.
 
 b. Haga lo mismo para una lente de Fresnel binaria de amplitud con los mismos parámetros constructivos.
 
 c. Idem para una lente de Fresnel de fase, de desfase $\pi$.
    
 d. Compare los 3 perfiles de intensidad.
 
 Para la visualización de los focos, haga un recorte de la imagen (plt.xlim y plt.ylim,  o cut_resample) para poder observarlo adecuadamente. También puede utilizar el parámetro logarithm de .draw() para resaltar los anillos.

2. Sea una lente de Fresnel binaria de fase, cuya focal es f′=10 mm y diámetro D=0.5 mm. Determine, en el marco XY y utilizando la aproximación de elemento delgado y el modelo de Rayleigh-Sommerfeld, la intensidad máxima en el foco en función del desfase entre las almenas de la lente, dentro del rango (0 - 2π).



3. Sea una red de Ronchi (con factor de relleno 0.5). Se pretende visualizar el efecto Talbot, mediante el formalismo XZ. Para ello se ilumina la red con una onda plana y se propaga mediante la función RS una distancia de varias veces la distancia de Talbot: $z_T = 2 p^2 / \lambda$, siendo p el periodo y $\lambda$ la longitud de onda (elegid los parámetros adecuados) para una correcta visualización. Como existe un efecto de borde al propagar, se puede recortar en el eje x, para eliminar dicho efecto de borde.

Proponga un valor para los parámetros que no vengan en el enunciado. En todos los casos, los resultados son gráficas de intensidad y fase luminosa.

## Resolución 1


In [ ]:
from dataclasses import dataclass, replace, field
from typing import Tuple, Callable
from diffractio.scalar_sources_XY import Scalar_source_XY
from diffractio.scalar_masks_XY import Scalar_mask_XY

from diffractio import degrees, mm, um, nm
from diffractio.scalar_fields_X import Scalar_field_X
from diffractio.scalar_sources_X import Scalar_source_X
from diffractio.scalar_masks_X import Scalar_mask_X

from matplotlib import pyplot as plt
import numpy as np

from numpy.typing import ArrayLike


@dataclass
class SimulationConfig:
    simulation_radius: float = 0.5 * mm
    simulation_resolution: int = 1024


def defaultMaskFactory(x, y):
    mask = Scalar_mask_XY(x, y)
    mask.one_level(1)
    return mask


@dataclass
class SimulationParams:
    mask_factory: Callable[[ArrayLike, ArrayLike], Scalar_mask_XY] = defaultMaskFactory
    wavelength: float = 600 * nm
    observation_distance: float = None  # defaults to the focal length
    focal: float = 10 * mm
    lens_diameter: float = 0.5 * mm

    def __post_init__(self):
        if self.observation_distance is None:
            self.observation_distance = self.focal


def simulate(params=SimulationParams(), config=SimulationConfig()):

    x0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    y0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    u1 = Scalar_source_XY(x=x0, y=y0, wavelength=params.wavelength)
    u1.plane_wave(A=1, theta=0, phi=0)

    mask = params.mask_factory(x0, y0)

    u2 = u1 * mask

    observation = u2.RS(
        params.observation_distance,
        new_field=True,
        verbose=True,
    )
    # Looking for the horizontal in our mesh that is closest to the y=0 axis. We can't guarantee our mesh touches it.
    j = np.argmin(np.abs(y0))
    field_slice = observation.u[:, j]

    field = Scalar_field_X(x0, 1)
    field.u = field_slice

    return observation


def x_line(
    field: Scalar_source_XY, f=lambda field: np.argmin(np.abs(field.y))
) -> Scalar_field_X:
    """Find a line

    Args:
        field (Scalar_source_XY): field
        f (Callable[[Scalar_source_XY], int]): Index of the line to take

    Returns:
        Scalar_field_X: y=constant line
    """
    j = f(field)
    field_slice = field.u[:, j]
    field = Scalar_field_X(field.x, 1)
    field.u = field_slice
    return field


def plot_evolution(params=SimulationParams(), config=SimulationConfig(), title=None):
    print(params, config)
    observation = simulate(params, config)

    draw_intensity_phase_XY(
        observation,
        title=f"{title or ''}Wave observed at {params.observation_distance/mm:.1f}mm (Observation window {config.simulation_radius/mm:.1f}mm)",
    )

    field_slice = x_line(observation, lambda field: np.argmin(np.abs(field.y)))
    field_slice.draw(logarithm=True)
    plt.title("Field Intensity across the y=0 line")
    plt.show()

### **a.** Lente estándar en aproximación TEA

In [ ]:
def lens_system(params: SimulationParams = SimulationParams()) -> SimulationParams:
    def lens_mask_factory(x, y):
        mask = Scalar_mask_XY(x=x, y=y, wavelength=params.wavelength)

        mask.lens((0, 0), focal=params.focal, radius=params.lens_diameter / 2, angle=0)
        return mask

    return replace(params, mask_factory=lens_mask_factory)


plot_evolution(lens_system(), config=SimulationConfig(simulation_resolution=500))

### b. Lente binaria de amplitud

In [ ]:
def fresnel_lens(
    *, params: SimulationParams = SimulationParams(), **kwargs
) -> SimulationParams:
    """Parameters for a simulation with a fresnel lens

    Args:
        params (SimulationParams, optional): Defaults to SimulationParams().
        params (**kwargs, optional): The rest of parameters are used with the Scalar_mask_XY.fresnel_lens.

    Returns:
        SimulationParams: Parameters to simulate with
    """

    def lens_mask_factory(x, y):
        mask = Scalar_mask_XY(x=x, y=y, wavelength=params.wavelength)
        args = {
            "r0": (0, 0),
            "focal": params.focal,
            "radius": params.lens_diameter / 2,
            "angle": 0,
        }
        args.update(kwargs)
        mask.fresnel_lens(**args)
        # phase changes = height changes modify phase and optimize.
        return mask

    return replace(params, mask_factory=lens_mask_factory)


plot_evolution(params=fresnel_lens())

### c. Lente binaria de fase

In [ ]:
plot_evolution(params=fresnel_lens(kind="phase", phase=180 * degrees))

# Fresnel transform. Fractionary fourier transform

### d. Comparación

Mientras que la lente estandard solo presenta un orden cero. Podemos ver como la de Fresnel tiene ciertas eficiencia que se pierde en su intensidad en anillos a 200 microas en ambas direcciones en el orden 1.

A la vez, la lente de fresnel de fase mantiene anillos de amplitudes muy similares, ya que estamos en fase cerca de zero.)

## Resolución 2

2. Sea una lente de Fresnel binaria de fase, cuya focal es f′=10 mm y diámetro D=0.5 mm. Determine, en el marco XY y utilizando la aproximación de elemento delgado y el modelo de Rayleigh-Sommerfeld, la intensidad máxima en el foco en función del desfase entre las almenas de la lente, dentro del rango (0 - 2π).


In [ ]:
from diffractio.scalar_fields_XY import Scalar_field_XY


def best_phase_search(params=SimulationParams(), config=SimulationConfig()):
    phases = range(360)
    solution = np.zeros((config.simulation_resolution, 360), dtype=complex)

    def get_max(field: Scalar_field_XY):
        (size, _) = field.u.shape
        return size // 2 + 1

    for i, alpha in enumerate(phases):

        # print(f"\nalpha={round(alpha * 360 / (2 * np.pi))}\n\t", )
        ## Possible BUG: Factor is printed with a \r character, so it is overwriting itself if you run multiple simulations
        observation = simulate(
            fresnel_lens(kind="phase", phase=alpha * np.pi / 180, params=params),
            config=config,
        )
        if alpha % 15 == 0:
            draw_intensity_phase_XY(
                observation,
                title=f"{alpha=}",
            )
            field_slice = x_line(observation, get_max)
            field_slice.draw(logarithm=True)
            plt.title(f"{alpha=}")
            plt.show()

        best_slice = x_line(observation, get_max)
        solution[:, i] = best_slice.u

    return solution, phases


## Possible bug:
solution, phases = best_phase_search(config=SimulationConfig(simulation_resolution=512))

In [ ]:
(psize, nangles) = solution.shape

plot_img = solution[psize // 4 : -psize // 4, :]
plot_img = np.abs(plot_img) + 1
# Normalize to 1
plot_img = plot_img / plot_img.max()
# Give full pixel color range, 0 to 255
plt.imshow(plot_img * 255, origin="lower")
plt.title(
    "Profile of simulations.\n Each $\\alpha$ corresponds to a different simulation",
)
plt.xlabel("$\\alpha$, phase of fresnel lense")
plt.ylabel("Intensity profile across X=0\n (observation plane)")
plt.show()
solution.shape

In [ ]:
# Maximum phase

max_intensity = np.amax(np.abs(solution) ** 2, axis=0)
plt.plot([*range(360)], max_intensity)
plt.title("Center intensity for each simulation")
plt.xlabel(r"$\alpha$")
plt.ylabel(r"Proportional to max intensity per simulation")
plt.show()

## Resolución 3

3. Sea una red de Ronchi (con factor de relleno 0.5). Se pretende visualizar el efecto Talbot, mediante el formalismo XZ. Para ello se ilumina la red con una onda plana y se propaga mediante la función RS una distancia de varias veces la distancia de Talbot: $z_T = 2 p^2 / \lambda$, siendo $p$ el periodo y $\lambda$ la longitud de onda (elegid los parámetros adecuados) para una correcta visualización. Como existe un efecto de borde al propagar, se puede recortar en el eje x, para eliminar dicho efecto de borde.

In [ ]:
# Red de ronchi:
## Red binaria, de amplitud con fill factor - 0.5
## Talbot Distance
##
@dataclass
class SimulationParams:
    wavelength: float = 600 * nm


@dataclass
class SimulationConfig:
    simulation_radius: float = 0.8 * mm
    simulation_max_z: float = 5 * mm
    simulation_resolution: int = 1024


def plot_ronchi_evolution(params=SimulationParams(), config=SimulationConfig()):
    wavelength = params.wavelength

    x0 = np.linspace(
        -config.simulation_radius,
        config.simulation_radius,
        config.simulation_resolution,
    )
    # Choose period so we have at least 10 pixels
    period = x0[17] - x0[0]

    z0 = np.linspace(-50 * um, config.simulation_max_z, config.simulation_resolution)

    u0 = Scalar_source_X(x=x0, wavelength=wavelength)
    u0.plane_wave()
    mask = Scalar_mask_X(x=x0, wavelength=wavelength)
    mask.ronchi_grating(0, period)
    u1 = mask * u0
    u1.draw()
    plt.title("Rochi grating")
    plt.plot()
    observation = Scalar_mask_XZ(x=x0, z=z0, wavelength=wavelength)
    observation.incident_field(u1)
    observation.WPM(has_edges=True)
    observation.draw()
    talbot_limit = 2 * period**2 / params.wavelength
    for i in range(3):
        plt.axvline(x=talbot_limit * i)
    plt.title(
        f"Rochi limit is {talbot_limit/um:.0f}um\n Blue lines display these planes"
    )
    plt.show()


plot_ronchi_evolution()
## Efecto talbot. Self-imaging at certain distances from the net when the net is periodic.
# This happens at z = 2 p^2/\lamba

# algorithm. Chirped z transform. You can go from 1025x1024 ot different sizes Algorithm

Por algun motivo, $z_T = 2 p^2 / \lambda$ parece que deja la mitad de los puntos en el que parece que la imagen inicial se recupera.

Seguramente esto se deba a que he hecho algún cálculo mal.